# Fine-Tuned Question Embedding Spider Experiments

In [ ]:
import os
import json
import nest_asyncio
from tqdm import tqdm
from utils.prompts.prompt_builder import prompt_factory
from utils.data.data_builder import load_data
from utils.llm.ask_llm import run_llm
from utils.data.post_process import save_results
from third_party.spider_eval.evaluation import evaluate_spider
from utils.prompts.prompt_builder import get_openai_key

EMBEDDING = "s2593817/sft-question-embedding"

In [1]:
from openai import OpenAI

# Set the environment variable to provide access to the OpenAI API (set in utils/parameters.py)
os.environ['OPENAI_API_KEY'] = get_openai_key()

# Create the OpenAI client
client = OpenAI()

In [ ]:
# Load the Spider dataset

path_data = "benchmarks"

data = load_data("spider", path_data)

## One-Shot Fine-Tuned Question Embedding Experiment

In [ ]:
# Instantiate a 1-shot sft-question-embedding prompt factory using Question Masked Euclidean Distance selection

prompt = prompt_factory(k_shot = 1, repr_type= "SQL", example_format= "QA", selector_type= "EUCDISTANCE", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 1-Shot GPT-3.5-Turbo Text-to-SQL conversion

questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    
    questions.append(question_format)

100%|██████████| 1034/1034 [03:52<00:00,  4.45it/s]


In [ ]:
# List the model parameters used across the experiment
args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 1,
    "example_type": "QA",
    "selector_type": "EUCDISQUESTION",
    "embedding_model": EMBEDDING
}

# Define the task dictionary to save the formatted questions and metadata of the Spider experiment
task = {
        "args": args,
        "questions": questions
    }

# Save the 1-shot sft-question-embedding Spider prompts to .json file for processing
OUT_DIR = os.path.join("appendicies","appendix-d-results","1-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-prompts-1.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [ ]:
# Send the formatted prompts to the GPT-3.5-TURBO model for response generation

OUT_DIR = os.path.join("appendicies","appendix-d-results","1-shot")

RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-1.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [12:50<00:00,  1.34it/s] 


In [ ]:
# Evaluate the generated GPT-3.5-Turbo responses and save results to file
nest_asyncio.apply()

OUT_DIR = os.path.join("appendicies","appendix-d-results","1-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sft-question-embedding-prompts-1.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-1.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-results-1.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.895                0.854                0.759                0.578                0.804               


## Three-Shot Fine-Tuned Question Embedding Experiment

In [ ]:
# Instantiate a 3-shot sft-question-embedding prompt factory using Question Masked Euclidean Distance selection

prompt = prompt_factory(k_shot = 3, repr_type= "SQL", example_format= "QA", selector_type= "EUCDISTANCE", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 1-Shot GPT-3.5-Turbo Text-to-SQL conversion

questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    
    questions.append(question_format)

100%|██████████| 1034/1034 [04:42<00:00,  3.66it/s]


In [ ]:
# List the model parameters used across the experiment
args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 3,
    "example_type": "QA",
    "selector_type": "EUCDISQUESTION",
    "embedding_model": EMBEDDING
}

# Define the task dictionary to save the formatted questions and metadata of the Spider experiment
task = {
        "args": args,
        "questions": questions
    }

# Save the 3-shot sft-question-embedding Spider prompts to .json file for processing
OUT_DIR = os.path.join("appendicies","appendix-d-results","3-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-prompts-3.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [ ]:
# Send the formatted prompts to the GPT-3.5-TURBO model for response generation

OUT_DIR = os.path.join("appendicies","appendix-d-results","3-shot")

RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-3.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [12:23<00:00,  1.39it/s]


In [ ]:
# Evaluate the generated GPT-3.5-Turbo responses and save results to file
nest_asyncio.apply()

OUT_DIR = os.path.join("appendicies","appendix-d-results","3-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sft-question-embedding-prompts-3.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-3.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-results-3.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.911                0.852                0.690                0.542                0.789               


## Five-Shot Fine-Tuned Question Embedding Experiment

In [ ]:
# Instantiate a 5-shot sft-question-embedding prompt factory using Question Masked Euclidean Distance selection

prompt = prompt_factory(k_shot = 5, repr_type= "SQL", example_format= "QA", selector_type= "EUCDISTANCE", embedding_model= EMBEDDING)(data=data, tokenizer="gpt-3.5-turbo")

In [ ]:
# Format all Spider test questions for 1-Shot GPT-3.5-Turbo Text-to-SQL conversion

questions = list()

for i, question_json in enumerate(tqdm(getattr(data, "get_test_json")()), start=1):
    
    question_format = prompt.format(index=i,
                                    target=question_json,
                                    max_seq_len=2048,
                                    max_ans_len=200,
                                    scope_factor=1,
                                    cross_domain=False)
    
    questions.append(question_format)

100%|██████████| 1034/1034 [05:15<00:00,  3.27it/s]


In [ ]:
# List the model parameters used across the experiment
args = {
    "data_type": "spider",
    "split": "test",
    "tokenizer": "gpt-3.5-turbo",
    "max_seq_length": 2048,
    "prompt_repr": "SQL",
    "k-shot": 5,
    "example_type": "QA",
    "selector_type": "EUCDISQUESTION",
    "embedding_model": EMBEDDING
}

# Define the task dictionary to save the formatted questions and metadata of the Spider experiment
task = {
        "args": args,
        "questions": questions
    }

# Save the 5-shot sft-question-embedding Spider prompts to .json file for processing
OUT_DIR = os.path.join("appendicies","appendix-d-results","5-shot")
os.makedirs(OUT_DIR, exist_ok =True)

PROMPTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-prompts-5.json")
json.dump(task, open(PROMPTS_FILE, "w"), indent=4)

In [ ]:
# Send the formatted prompts to the GPT-3.5-TURBO model for response generation

OUT_DIR = os.path.join("appendicies","appendix-d-results","5-shot")

RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-5.txt")

run_llm(PROMPTS_FILE, RESPONSES_FILE, model="gpt-3.5-turbo")

100%|██████████| 1034/1034 [12:22<00:00,  1.39it/s]


In [ ]:
# Evaluate the generated GPT-3.5-Turbo responses and save results to file
nest_asyncio.apply()

OUT_DIR = os.path.join("appendicies","appendix-d-results","5-shot")

PROMPTS_FILE = os.path.join(OUT_DIR, "sft-question-embedding-prompts-5.json")
RESPONSES_FILE = os.path.join(OUT_DIR, "sft-question-embedding-responses-5.txt")
RESULTS_FILE = os.path.join(OUT_DIR,"sft-question-embedding-results-5.json")

results = evaluate_spider(gold="benchmarks/spider/dev_gold.sql", pred=RESPONSES_FILE, db="benchmarks/spider/databases", table="benchmarks/spider/tables.json")

save_results(PROMPTS_FILE, RESPONSES_FILE, RESULTS_FILE, results)

  0%|          | 0/1 [00:00<?, ?it/s]


                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.911                0.850                0.690                0.548                0.789               
